- Shenario: Will ask a question to llm1. if respnose length is greter then 300 we will summerize the contain using llm2 otherwise will run in llm3

In [1]:
import os
from dotenv import load_dotenv

load_dotenv('./../.env')

# Debug: Print to verify values are loaded
print(f"Endpoint: {os.getenv('LANGSMITH_ENDPOINT')}")
from langchain_ollama import ChatOllama

llm1 = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen2.5:latest",
    temperature=0.7,
    max_tokens=1024
)
llm2 = ChatOllama(
    base_url="http://localhost:11434",
    model="mistral:latest",
    temperature=0.7,
    max_tokens=1024
)
llm3 = ChatOllama(
    base_url="http://localhost:11434",
    model="DeepSeek-R1",
    temperature=0.7,
    max_tokens=1024
)

Endpoint: https://api.smith.langchain.com


In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel

prompt_template1 = ChatPromptTemplate([
    ("system","You are an LLM expert"),
    ("user","Why sky is blue?")])

chain1 = prompt_template1 | llm1 | StrOutputParser()
response = chain1.invoke({}) 
print("Response:"+response)

def choose_llm(response): 
    response_text = str(response)
    if len(response_text) > 100:
        return llm2
    else:
        return llm1

llm_selector = RunnableLambda(choose_llm)

prompt_template2 = ChatPromptTemplate([
    ("system","You are an LLM expert"),
    ("user","summerise {response} in one line.")])

chain2 = prompt_template2 | llm_selector | StrOutputParser()
answer = chain2.invoke({"response": response})
print("Answer:"+answer)

Response:The sky appears blue during the day due to a phenomenon called Rayleigh scattering. Here's a more detailed explanation:

1. **Sunlight Composition**: Sunlight contains all colors of the visible spectrum—red, orange, yellow, green, blue, indigo, and violet. When sunlight enters Earth's atmosphere, it interacts with molecules and small particles in the air.

2. **Scattering by Molecules**: The shorter, bluer wavelengths of light are scattered more easily by the gases and tiny particles (such as nitrogen and oxygen) in the atmosphere. This process is known as Rayleigh scattering.

3. **Blue Light Dominance**: Because blue light is scattered more than other colors, it becomes predominant in the sky when we look up at it during the day.

4. **Color Perception**: When you look at the sky directly, your eyes receive a mixture of these scattered blue wavelengths. Since your eye has a higher sensitivity to blue light and the sky doesn't have any other color to contrast with (like it do